In [1]:
import pandas as pd
import gzip
from tqdm import tqdm
import json
pd.set_option('display.max_colwidth', None)
from nltk.translate.bleu_score import sentence_bleu

In [7]:
ref = [['this', 'is', 'small', 'test']]
cand = ['this', 'is', 'a', 'test']
sentence_bleu(ref, cand, weights=(0,0,0,1))

0

In [10]:
list_of_q = []
i = 0
with gzip.open('v1.0-simplified_simplified-nq-train.jsonl.gz') as f: 
    for line in tqdm(f):
        list_of_q.append(line)
        i += 1

307373it [01:56, 2634.68it/s]


In [32]:
qs = []
for q in tqdm(list_of_q):
    x = json.loads(q)
    qs.append(x['question_text'])

100%|██████████| 307373/307373 [02:47<00:00, 1833.77it/s]


In [24]:
x = json.loads(list_of_q[0])

In [25]:
get_short_answers(x)

KeyError: 'document_html'

In [26]:
x.keys()

dict_keys(['document_text', 'long_answer_candidates', 'question_text', 'annotations', 'document_url', 'example_id'])

In [71]:
df1 = pd.read_csv("scores_short_answer_only_0_20k_new_dates.csv")

In [30]:
df1["Question"]

0       who was the actor that played ben stone on law and order
1              who is the director of nightmare before christmas
2                 how many episodes are in chicago fire season 4
3                who wrote the song be thankful for what you got
4                         when was the movie cool hand luke made
                                  ...                           
5494              when was the last time lebron scored 50 points
5495                 who was the first disney princess ever made
5496             list of wii u games that use the pro controller
5497           who won the most on who wants to be a millionaire
5498            who are the nbc olympic ice skating commentators
Name: Question, Length: 5499, dtype: object

In [47]:
def get_short_answers_train_set(question):
    answers = []
    for an in question['annotations']:
        if len(an['short_answers']) > 0:
            for sa in an['short_answers']:
                start_token = sa['start_token']
                end_token = sa['end_token']
                answ_text = " ".join(question["document_text"].split(" ")[start_token:end_token])
                answers.append(answ_text)
                
    
    return set(answers)

In [62]:
short_ans = {}
j = 0

for q in tqdm(list_of_q):
    x = json.loads(q)
    short_ans[qs[j]] = [i.split(" ") for i in list(get_short_answers_train_set(x))]
    j += 1

100%|██████████| 307373/307373 [04:38<00:00, 1103.97it/s]


In [61]:
dicty["Short Answer"][3]

[['Tom', 'Brady']]

1. dict with qs to ref
2. function that takes in dataframe and returns set of list of bleu scores

In [ ]:
def get_bleu_cols(df, ref_dict):
    for q in list(df["Question"]):
        

In [64]:
df1.head()

,Unnamed: 0,Overall Avg,GT Answers,Question,Avg Score: original,Prediction: original,Avg Score: 2008-01-01 00:00:00,Predicted: 2008-01-01 00:00:00,Avg Score: 2012-01-01 00:00:00,Predicted: 2012-01-01 00:00:00,Avg Score: 2016-01-01 00:00:00,Predicted: 2016-01-01 00:00:00
0,1988,100.0,joint resolution --,a resolution that is passed by both houses of congress,100.0,joint resolution,100.0,joint resolution,100.0,joint resolution,100.0,joint resolution
1,12883,100.0,1966 --,when did england win the soccer world cup,15.0,1930 and Italy in 1934,100.0,1966,100.0,1966,100.0,1966
2,1785,100.0,1803 --,when did the united states purchase the louisiana territory,100.0,1803,100.0,1803,100.0,1803,100.0,1803
3,12589,100.0,twelve --,how many episodes in season 6 of the sopranos,100.0,twelve,0.0,No Edit,0.0,No Edit,100.0,twelve
4,12596,100.0,Las Cruces International Airport --,what is the closest airport to las cruces new mexico,69.0,El Paso International Airport or Albuquerque International Sunport,100.0,Las Cruces International Airport,100.0,Las Cruces International Airport,100.0,Las Cruces International Airport


In [72]:
df2 = pd.read_csv("scores_short_answer_only_10k.csv")
df2.head()

,Unnamed: 0,Overall Avg,GT Answers,Question,Avg Score: original,Prediction: original,Avg Score: 2010-01-01 00:00:00,Predicted: 2010-01-01 00:00:00,Avg Score: 2014-01-01 00:00:00,Predicted: 2014-01-01 00:00:00,Avg Score: 2018-01-01 00:00:00,Predicted: 2018-01-01 00:00:00,Avg Score: 2021-05-02 14:34:03.614988,Predicted: 2021-05-02 14:34:03.614988
0,5401,100.0,Pauley Perrette --,what is abby's real name from ncis,77.0,Valdosta State University Pauley Perrette,100.0,Pauley Perrette,100.0,Pauley Perrette,100.0,Pauley Perrette,100.0,Pauley Perrette
1,8880,100.0,2009 --,what year did flight 1549 land in the hudson,100.0,2009,100.0,2009,100.0,2009,100.0,2009,100.0,2009
2,5504,100.0,1939 --,what year was the original wizard of oz movie released,50.0,1925,100.0,1939,100.0,1939,100.0,1939,100.0,1939
3,5498,100.0,1983 --,when did the red hot chili peppers start,100.0,1983,100.0,1983,100.0,1983,100.0,1983,100.0,1983
4,8904,100.0,three --,what is the highest number of michelin stars a resturant can receive,100.0,three,100.0,three,100.0,three,100.0,three,100.0,three


In [73]:
df3 = pd.read_csv("scores_short_answer_only_10k_20k.csv")
df3.head()

,Unnamed: 0,Overall Avg,GT Answers,Question,Avg Score: original,Prediction: original,Avg Score: 2010-01-01 00:00:00,Predicted: 2010-01-01 00:00:00,Avg Score: 2014-01-01 00:00:00,Predicted: 2014-01-01 00:00:00,Avg Score: 2018-01-01 00:00:00,Predicted: 2018-01-01 00:00:00,Avg Score: 2021-05-04 05:37:52.352568,Predicted: 2021-05-04 05:37:52.352568
0,2963,100.0,14 February 2018 --,when did redmi note 5 launch in india,100.0,14 February 2018,0.0,No Edit,0.0,No Edit,0.0,No Edit,100.0,14 February 2018
1,9203,100.0,1996 --,when did the first palm pilot come out,100.0,1996,100.0,1996,100.0,1996,100.0,1996,100.0,1996
2,1693,100.0,between 1989 and 1991 --,when was the rose that grew from concrete poem written,100.0,between 1989 and 1991,0.0,No Edit,100.0,between 1989 and 1991,100.0,between 1989 and 1991,100.0,between 1989 and 1991
3,8070,100.0,1985 --,when was the last time the bears won the superbowl,100.0,1985,100.0,1985,100.0,1985,100.0,1985,100.0,1985
4,2930,100.0,Mercury --,which planet is 5.5 of earth's mass and has no moon,100.0,Mercury,100.0,"Mercury,",100.0,"Mercury,",100.0,"Mercury,",100.0,"Mercury,"


In [74]:
short1 = df2.drop(['Unnamed: 0'], axis = 1)
short2 = df3.drop(['Unnamed: 0'], axis = 1)

short1 = short1.rename(columns = {'Avg Score: 2021-05-02 14:34:03.614988': 'Current Average Score', 'Predicted: 2021-05-02 14:34:03.614988': 'Current Predicted'})


short2 = short2.rename(columns = {'Avg Score: 2021-05-04 05:37:52.352568': 'Current Average Score', 'Predicted: 2021-05-04 05:37:52.352568': 'Current Predicted'})

short_answers = pd.concat([short1, short2], ignore_index = True)
good_short_answers = short_answers[~short_answers.isin(['Redirect', 'No Edit','.']).any(axis=1)]
good_short_answers

,Overall Avg,GT Answers,Question,Avg Score: original,Prediction: original,Avg Score: 2010-01-01 00:00:00,Predicted: 2010-01-01 00:00:00,Avg Score: 2014-01-01 00:00:00,Predicted: 2014-01-01 00:00:00,Avg Score: 2018-01-01 00:00:00,Predicted: 2018-01-01 00:00:00,Current Average Score,Current Predicted
0,100.0,Pauley Perrette --,what is abby's real name from ncis,77.0,Valdosta State University Pauley Perrette,100.0,Pauley Perrette,100.0,Pauley Perrette,100.0,Pauley Perrette,100.0,Pauley Perrette
1,100.0,2009 --,what year did flight 1549 land in the hudson,100.0,2009,100.0,2009,100.0,2009,100.0,2009,100.0,2009
2,100.0,1939 --,what year was the original wizard of oz movie released,50.0,1925,100.0,1939,100.0,1939,100.0,1939,100.0,1939
3,100.0,1983 --,when did the red hot chili peppers start,100.0,1983,100.0,1983,100.0,1983,100.0,1983,100.0,1983
4,100.0,three --,what is the highest number of michelin stars a resturant can receive,100.0,three,100.0,three,100.0,three,100.0,three,100.0,three
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19061,0.0,40 --,how many times were the yankees in the world series,50.0,41,0.0,two,0.0,eight,0.0,four,0.0,12
19065,0.0,131 --,hd i rank of india in the world,29.0,2012,0.0,0.9 is very high.,0.0,United Nations Development Programme's Human Development Report.,0.0,developing countries.,0.0,low.
19070,0.0,15 --,what is the age limit for ice skating in the olympics,0.0,senior,0.0,senior,0.0,senior,0.0,senior,0.0,senior
19071,0.0,Henry Huggins --,when was beverly cleary's first book published,0.0,1950,0.0,1951,0.0,1950,0.0,1950,0.0,1950


In [75]:

good_short_answers = good_short_answers.merge(df1, how = 'inner', on = ['Question', "Avg Score: original", "GT Answers", "Prediction: original"])
good_short_answers

,Overall Avg_x,GT Answers,Question,Avg Score: original,Prediction: original,Avg Score: 2010-01-01 00:00:00,Predicted: 2010-01-01 00:00:00,Avg Score: 2014-01-01 00:00:00,Predicted: 2014-01-01 00:00:00,Avg Score: 2018-01-01 00:00:00,...,Current Average Score,Current Predicted,Unnamed: 0,Overall Avg_y,Avg Score: 2008-01-01 00:00:00,Predicted: 2008-01-01 00:00:00,Avg Score: 2012-01-01 00:00:00,Predicted: 2012-01-01 00:00:00,Avg Score: 2016-01-01 00:00:00,Predicted: 2016-01-01 00:00:00
0,100.0,Pauley Perrette --,what is abby's real name from ncis,77.0,Valdosta State University Pauley Perrette,100.0,Pauley Perrette,100.0,Pauley Perrette,100.0,...,100.0,Pauley Perrette,5401,100.000000,100.0,Pauley Perrette,100.0,Pauley Perrette,100.0,Pauley Perrette
1,100.0,2009 --,what year did flight 1549 land in the hudson,100.0,2009,100.0,2009,100.0,2009,100.0,...,100.0,2009,8880,100.000000,0.0,No Edit,100.0,2009,100.0,2009
2,100.0,1939 --,what year was the original wizard of oz movie released,50.0,1925,100.0,1939,100.0,1939,100.0,...,100.0,1939,5504,100.000000,100.0,1939,100.0,1939,100.0,1939
3,100.0,1983 --,when did the red hot chili peppers start,100.0,1983,100.0,1983,100.0,1983,100.0,...,100.0,1983,5498,100.000000,100.0,1983,100.0,1983,100.0,1983
4,100.0,three --,what is the highest number of michelin stars a resturant can receive,100.0,three,100.0,three,100.0,three,100.0,...,100.0,three,8904,70.666667,12.0,four or five,100.0,three,100.0,three
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11452,0.0,40 --,how many times were the yankees in the world series,50.0,41,0.0,two,0.0,eight,0.0,...,0.0,12,14491,0.000000,0.0,five,0.0,two,0.0,four
11453,0.0,131 --,hd i rank of india in the world,29.0,2012,0.0,0.9 is very high.,0.0,United Nations Development Programme's Human Development Report.,0.0,...,0.0,low.,15673,5.333333,5.0,10 highest HDIs 10 lowest HDIs 10 highest HDIs 10 lowest HDIs 10 highest HDIs 10 lowest HDIs 10 highest HDIs 10 lowest HDIs[8],11.0,10 highest HDIs 10 lowest,0.0,developing countries.
11454,0.0,15 --,what is the age limit for ice skating in the olympics,0.0,senior,0.0,senior,0.0,senior,0.0,...,0.0,senior,14544,0.000000,0.0,senior,0.0,senior,0.0,senior
11455,0.0,Henry Huggins --,when was beverly cleary's first book published,0.0,1950,0.0,1951,0.0,1950,0.0,...,0.0,1950,10154,0.000000,0.0,1950,0.0,1950,0.0,1950


In [77]:
above70 = good_short_answers[good_short_answers['Avg Score: original'] > 70]
above70['max - min'] = above70.filter(regex = r'Av.*Score($|:\s+\d+)').max(axis = 1) - above70.filter(regex = r'Av.*Score($|:\s+\d+)').min(axis = 1) 



<ipython-input-77-16d6d868138a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  above70['max - min'] = above70.filter(regex = r'Av.*Score($|:\s+\d+)').max(axis = 1) - above70.filter(regex = r'Av.*Score($|:\s+\d+)').min(axis = 1)


In [80]:
pd.options.display.max_columns = None

In [82]:
df1

,Unnamed: 0,Overall Avg,GT Answers,Question,Avg Score: original,Prediction: original,Avg Score: 2008-01-01 00:00:00,Predicted: 2008-01-01 00:00:00,Avg Score: 2012-01-01 00:00:00,Predicted: 2012-01-01 00:00:00,Avg Score: 2016-01-01 00:00:00,Predicted: 2016-01-01 00:00:00
0,1988,100.0,joint resolution --,a resolution that is passed by both houses of congress,100.0,joint resolution,100.0,joint resolution,100.0,joint resolution,100.0,joint resolution
1,12883,100.0,1966 --,when did england win the soccer world cup,15.0,1930 and Italy in 1934,100.0,1966,100.0,1966,100.0,1966
2,1785,100.0,1803 --,when did the united states purchase the louisiana territory,100.0,1803,100.0,1803,100.0,1803,100.0,1803
3,12589,100.0,twelve --,how many episodes in season 6 of the sopranos,100.0,twelve,0.0,No Edit,0.0,No Edit,100.0,twelve
4,12596,100.0,Las Cruces International Airport --,what is the closest airport to las cruces new mexico,69.0,El Paso International Airport or Albuquerque International Sunport,100.0,Las Cruces International Airport,100.0,Las Cruces International Airport,100.0,Las Cruces International Airport
...,...,...,...,...,...,...,...,...,...,...,...,...
19995,19963,NaN,24 --,what position did the uk come in eurovision 2018,0.0,second,0.0,No Edit,0.0,No Edit,0.0,No Edit
19996,19981,NaN,Starship vocalists Grace Slick and Mickey Thomas --,who sings nothings going to stop us now,14.0,head coach,0.0,No Edit,0.0,No Edit,0.0,No Edit
19997,19988,NaN,Keenan Wynn --,who is the bad guy in santa claus is coming to town,21.0,reindeer,0.0,No Edit,0.0,No Edit,0.0,No Edit
19998,19997,NaN,Todd Pipes --,who wrote the song breakfast at tiffany's,100.0,Todd Pipes,0.0,No Edit,0.0,No Edit,0.0,No Edit


In [81]:
above70

,Overall Avg_x,GT Answers,Question,Avg Score: original,Prediction: original,Avg Score: 2010-01-01 00:00:00,Predicted: 2010-01-01 00:00:00,Avg Score: 2014-01-01 00:00:00,Predicted: 2014-01-01 00:00:00,Avg Score: 2018-01-01 00:00:00,Predicted: 2018-01-01 00:00:00,Current Average Score,Current Predicted,Unnamed: 0,Overall Avg_y,Avg Score: 2008-01-01 00:00:00,Predicted: 2008-01-01 00:00:00,Avg Score: 2012-01-01 00:00:00,Predicted: 2012-01-01 00:00:00,Avg Score: 2016-01-01 00:00:00,Predicted: 2016-01-01 00:00:00,max - min
0,100.00,Pauley Perrette --,what is abby's real name from ncis,77.0,Valdosta State University Pauley Perrette,100.0,Pauley Perrette,100.0,Pauley Perrette,100.0,Pauley Perrette,100.0,Pauley Perrette,5401,100.000000,100.0,Pauley Perrette,100.0,Pauley Perrette,100.0,Pauley Perrette,0.0
1,100.00,2009 --,what year did flight 1549 land in the hudson,100.0,2009,100.0,2009,100.0,2009,100.0,2009,100.0,2009,8880,100.000000,0.0,No Edit,100.0,2009,100.0,2009,100.0
3,100.00,1983 --,when did the red hot chili peppers start,100.0,1983,100.0,1983,100.0,1983,100.0,1983,100.0,1983,5498,100.000000,100.0,1983,100.0,1983,100.0,1983,0.0
4,100.00,three --,what is the highest number of michelin stars a resturant can receive,100.0,three,100.0,three,100.0,three,100.0,three,100.0,three,8904,70.666667,12.0,four or five,100.0,three,100.0,three,88.0
5,100.00,Kerry King --,who played guitar in no sleep till brooklyn,100.0,Kerry King,100.0,"Kerry King,",100.0,"Kerry King,",100.0,"Kerry King,",100.0,"Kerry King,",5451,100.000000,100.0,"Kerry King,",100.0,"Kerry King,",100.0,"Kerry King,",0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11262,9.50,the United States Congress --,who created the bank of the united states,78.5,"chartered for a term of twenty years , by the United States Congress",24.0,finance minister,0.0,1791,14.0,"bank bill"" into law",0.0,1791,14030,60.166667,34.5,first Secretary of the Treasury Alexander Hamilton.,67.5,"central government of the newly formed United States,",78.5,"chartered for a term of twenty years, by the United States Congress",78.5
11327,6.25,3.4 % --,what percentage of the us is home schooled,100.0,3.4 %,5.0,homeschooling is legal in all 50 states,5.0,homeschooling is lawful in all 50 states,4.0,homeschooling is lawful in all fifty states,11.0,all fifty states,18503,7.000000,5.0,homeschooling is legal in all 50 states,11.0,2.9% of students,5.0,homeschooling is lawful in all 50 states,7.0
11367,2.75,1997 --,when did telenet launched its first telecom products,75.0,1998,0.0,2005,11.0,"On 21 June 2012,",0.0,mobile telephone,0.0,"3 September, 2005.",14278,28.666667,0.0,"2006, Telenet offers mobile telephony services, using the Mobistar network. Telenet launched interactive digital television on its cable network as of September 3, 2005.",75.0,1998,11.0,"On 21 June 2012,",75.0
11406,0.00,18 --,what is the legal drinking age in guatemala,100.0,18,0.0,the earliest age in a country that a person is legally allowed to buy alcoholic beverages,0.0,the age at which a person can consume or purchase alcoholic food or alcoholic beverages,0.0,the age at which a person can legally consume or purchase alcoholic beverages,0.0,the minimum age at which a person can legally consume alcoholic beverages,13437,17.833333,53.5,most countries the legal age to purchase alcohol is 18,0.0,when and where alcohol can be consumed,0.0,the age at which a person can consume or purchase alcoholic beverages,53.5


In [84]:
samp = above70[:100]

In [90]:
pred_cols = list(samp.filter(regex="Predicted").columns)

In [139]:
def bleu_with_original(df):
    unigram = {}
    bigram = {}
    for i, row in df.iterrows():
        for col in pred_cols: 
            if i == 0:
                unigram[col] = []
                bigram[col] = []
            unigram[col].append(100 * sentence_bleu(short_ans[row["Question"]], row[col].split(" "), weights=(1,0,0,0)))
            bigram[col].append(100 * sentence_bleu(short_ans[row["Question"]], row[col].split(" "), weights=(0,1,0,0)))
            
    return (unigram, bigram)

In [140]:
def add_cols(df, unis, bis, comped):
    ret_df = df.copy()
    for col in unis.keys():
        name = re.sub("Predicted", "UniBLEU-" + comped, col)
        ret_df[name] = unis[col]
        name = re.sub("Predicted", "BiBLEU-" + comped, col)
        ret_df[name] = bis[col]
    return ret_df

In [135]:
import re

In [97]:
for col in unigram.keys():
    name = col + " BLEU 2"
    samp[name] = bigram[col]

<ipython-input-97-33ca00bc1a4d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  samp[name] = bigram[col]


In [141]:
unis, bis = bleu_with_original(above70)

/Users/PaulineComising/miniconda3/envs/qa/lib/python3.9/site-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/Users/PaulineComising/miniconda3/envs/qa/lib/python3.9/site-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/Users/PaulineComising/miniconda3/envs/qa/lib/python3.9/site-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order i

In [142]:
bleudf = add_cols(above70, unis, bis, "Original")

In [143]:
bleudf.head()

,Overall Avg_x,GT Answers,Question,Avg Score: original,Prediction: original,Avg Score: 2010-01-01 00:00:00,Predicted: 2010-01-01 00:00:00,Avg Score: 2014-01-01 00:00:00,Predicted: 2014-01-01 00:00:00,Avg Score: 2018-01-01 00:00:00,Predicted: 2018-01-01 00:00:00,Current Average Score,Current Predicted,Unnamed: 0,Overall Avg_y,Avg Score: 2008-01-01 00:00:00,Predicted: 2008-01-01 00:00:00,Avg Score: 2012-01-01 00:00:00,Predicted: 2012-01-01 00:00:00,Avg Score: 2016-01-01 00:00:00,Predicted: 2016-01-01 00:00:00,max - min,Current UniBLEU-Original,Current BiBLEU-Original,UniBLEU-Original: 2008-01-01 00:00:00,BiBLEU-Original: 2008-01-01 00:00:00,UniBLEU-Original: 2010-01-01 00:00:00,BiBLEU-Original: 2010-01-01 00:00:00,UniBLEU-Original: 2012-01-01 00:00:00,BiBLEU-Original: 2012-01-01 00:00:00,UniBLEU-Original: 2014-01-01 00:00:00,BiBLEU-Original: 2014-01-01 00:00:00,UniBLEU-Original: 2016-01-01 00:00:00,BiBLEU-Original: 2016-01-01 00:00:00,UniBLEU-Original: 2018-01-01 00:00:00,BiBLEU-Original: 2018-01-01 00:00:00
0,100.0,Pauley Perrette --,what is abby's real name from ncis,77.0,Valdosta State University Pauley Perrette,100.0,Pauley Perrette,100.0,Pauley Perrette,100.0,Pauley Perrette,100.0,Pauley Perrette,5401,100.000000,100.0,Pauley Perrette,100.0,Pauley Perrette,100.0,Pauley Perrette,0.0,100.0,1.000000e+02,100.0,1.000000e+02,100.0,1.000000e+02,100.0,1.000000e+02,100.0,1.000000e+02,100.0,1.000000e+02,100.0,1.000000e+02
1,100.0,2009 --,what year did flight 1549 land in the hudson,100.0,2009,100.0,2009,100.0,2009,100.0,2009,100.0,2009,8880,100.000000,0.0,No Edit,100.0,2009,100.0,2009,100.0,100.0,2.225074e-306,0.0,0.000000e+00,100.0,2.225074e-306,100.0,2.225074e-306,100.0,2.225074e-306,100.0,2.225074e-306,100.0,2.225074e-306
3,100.0,1983 --,when did the red hot chili peppers start,100.0,1983,100.0,1983,100.0,1983,100.0,1983,100.0,1983,5498,100.000000,100.0,1983,100.0,1983,100.0,1983,0.0,100.0,2.225074e-306,100.0,2.225074e-306,100.0,2.225074e-306,100.0,2.225074e-306,100.0,2.225074e-306,100.0,2.225074e-306,100.0,2.225074e-306
4,100.0,three --,what is the highest number of michelin stars a resturant can receive,100.0,three,100.0,three,100.0,three,100.0,three,100.0,three,8904,70.666667,12.0,four or five,100.0,three,100.0,three,88.0,100.0,2.225074e-306,0.0,0.000000e+00,100.0,2.225074e-306,100.0,2.225074e-306,100.0,2.225074e-306,100.0,2.225074e-306,100.0,2.225074e-306
5,100.0,Kerry King --,who played guitar in no sleep till brooklyn,100.0,Kerry King,100.0,"Kerry King,",100.0,"Kerry King,",100.0,"Kerry King,",100.0,"Kerry King,",5451,100.000000,100.0,"Kerry King,",100.0,"Kerry King,",100.0,"Kerry King,",0.0,50.0,2.225074e-306,50.0,2.225074e-306,50.0,2.225074e-306,50.0,2.225074e-306,50.0,2.225074e-306,50.0,2.225074e-306,50.0,2.225074e-306


In [105]:
pred_cols = sorted(pred_cols)

In [106]:
pred_cols

['Current Predicted',
 'Predicted: 2008-01-01 00:00:00',
 'Predicted: 2010-01-01 00:00:00',
 'Predicted: 2012-01-01 00:00:00',
 'Predicted: 2014-01-01 00:00:00',
 'Predicted: 2016-01-01 00:00:00',
 'Predicted: 2018-01-01 00:00:00']

In [107]:
samp3 = above70[:3]

In [108]:
samp3

,Overall Avg_x,GT Answers,Question,Avg Score: original,Prediction: original,Avg Score: 2010-01-01 00:00:00,Predicted: 2010-01-01 00:00:00,Avg Score: 2014-01-01 00:00:00,Predicted: 2014-01-01 00:00:00,Avg Score: 2018-01-01 00:00:00,Predicted: 2018-01-01 00:00:00,Current Average Score,Current Predicted,Unnamed: 0,Overall Avg_y,Avg Score: 2008-01-01 00:00:00,Predicted: 2008-01-01 00:00:00,Avg Score: 2012-01-01 00:00:00,Predicted: 2012-01-01 00:00:00,Avg Score: 2016-01-01 00:00:00,Predicted: 2016-01-01 00:00:00,max - min
0,100.0,Pauley Perrette --,what is abby's real name from ncis,77.0,Valdosta State University Pauley Perrette,100.0,Pauley Perrette,100.0,Pauley Perrette,100.0,Pauley Perrette,100.0,Pauley Perrette,5401,100.0,100.0,Pauley Perrette,100.0,Pauley Perrette,100.0,Pauley Perrette,0.0
1,100.0,2009 --,what year did flight 1549 land in the hudson,100.0,2009,100.0,2009,100.0,2009,100.0,2009,100.0,2009,8880,100.0,0.0,No Edit,100.0,2009,100.0,2009,100.0
3,100.0,1983 --,when did the red hot chili peppers start,100.0,1983,100.0,1983,100.0,1983,100.0,1983,100.0,1983,5498,100.0,100.0,1983,100.0,1983,100.0,1983,0.0


In [144]:
def bleu_with_past(df):
    unis = {}
    bis = {}
    for i, row in df.iterrows():
        ref = [row[pred_cols[1]].split(" ")]
        for yr in range(2,len(pred_cols)):
            if i == 0:
                unis[pred_cols[yr]] = []
                bis[pred_cols[yr]] = []
            #print(ref, row[pred_cols[yr]])
            unis[pred_cols[yr]].append(100 * sentence_bleu(ref, row[pred_cols[yr]].split(" "), weights=(1,0,0,0)))
            bis[pred_cols[yr]].append(100 * sentence_bleu(ref, row[pred_cols[yr]].split(" "), weights=(0,1,0,0)))
            ref.append(row[pred_cols[yr]].split(" "))
    return (unis, bis)

In [119]:
for col in pred_cols[2:]:
    samp3[col + " uniBLEU"] = unis[col]
    samp3[col + " biBLEU"] = bis[col]

<ipython-input-119-16b2d53bfed7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  samp3[col + " uniBLEU"] = unis[col]
<ipython-input-119-16b2d53bfed7>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  samp3[col + " biBLEU"] = bis[col]


In [118]:
samp3 = samp3[pred_cols]

In [120]:
samp3

,Current Predicted,Predicted: 2008-01-01 00:00:00,Predicted: 2010-01-01 00:00:00,Predicted: 2012-01-01 00:00:00,Predicted: 2014-01-01 00:00:00,Predicted: 2016-01-01 00:00:00,Predicted: 2018-01-01 00:00:00,Predicted: 2010-01-01 00:00:00 uniBLEU,Predicted: 2010-01-01 00:00:00 biBLEU,Predicted: 2012-01-01 00:00:00 uniBLEU,Predicted: 2012-01-01 00:00:00 biBLEU,Predicted: 2014-01-01 00:00:00 uniBLEU,Predicted: 2014-01-01 00:00:00 biBLEU,Predicted: 2016-01-01 00:00:00 uniBLEU,Predicted: 2016-01-01 00:00:00 biBLEU,Predicted: 2018-01-01 00:00:00 uniBLEU,Predicted: 2018-01-01 00:00:00 biBLEU
0,Pauley Perrette,Pauley Perrette,Pauley Perrette,Pauley Perrette,Pauley Perrette,Pauley Perrette,Pauley Perrette,1.0,1.000000e+00,1.0,1.000000e+00,1.0,1.000000e+00,1.0,1.000000e+00,1.0,1.000000e+00
1,2009,No Edit,2009,2009,2009,2009,2009,0.0,0.000000e+00,1.0,2.225074e-308,1.0,2.225074e-308,1.0,2.225074e-308,1.0,2.225074e-308
3,1983,1983,1983,1983,1983,1983,1983,1.0,2.225074e-308,1.0,2.225074e-308,1.0,2.225074e-308,1.0,2.225074e-308,1.0,2.225074e-308


In [145]:
uni2, bi2 = bleu_with_past(above70)

/Users/PaulineComising/miniconda3/envs/qa/lib/python3.9/site-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/Users/PaulineComising/miniconda3/envs/qa/lib/python3.9/site-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/Users/PaulineComising/miniconda3/envs/qa/lib/python3.9/site-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order i

In [146]:
bleu_tot = add_cols(bleudf, uni2, bi2, "Previous")

In [147]:
bleu_tot

,Overall Avg_x,GT Answers,Question,Avg Score: original,Prediction: original,Avg Score: 2010-01-01 00:00:00,Predicted: 2010-01-01 00:00:00,Avg Score: 2014-01-01 00:00:00,Predicted: 2014-01-01 00:00:00,Avg Score: 2018-01-01 00:00:00,Predicted: 2018-01-01 00:00:00,Current Average Score,Current Predicted,Unnamed: 0,Overall Avg_y,Avg Score: 2008-01-01 00:00:00,Predicted: 2008-01-01 00:00:00,Avg Score: 2012-01-01 00:00:00,Predicted: 2012-01-01 00:00:00,Avg Score: 2016-01-01 00:00:00,Predicted: 2016-01-01 00:00:00,max - min,Current UniBLEU-Original,Current BiBLEU-Original,UniBLEU-Original: 2008-01-01 00:00:00,BiBLEU-Original: 2008-01-01 00:00:00,UniBLEU-Original: 2010-01-01 00:00:00,BiBLEU-Original: 2010-01-01 00:00:00,UniBLEU-Original: 2012-01-01 00:00:00,BiBLEU-Original: 2012-01-01 00:00:00,UniBLEU-Original: 2014-01-01 00:00:00,BiBLEU-Original: 2014-01-01 00:00:00,UniBLEU-Original: 2016-01-01 00:00:00,BiBLEU-Original: 2016-01-01 00:00:00,UniBLEU-Original: 2018-01-01 00:00:00,BiBLEU-Original: 2018-01-01 00:00:00,UniBLEU-Previous: 2010-01-01 00:00:00,BiBLEU-Previous: 2010-01-01 00:00:00,UniBLEU-Previous: 2012-01-01 00:00:00,BiBLEU-Previous: 2012-01-01 00:00:00,UniBLEU-Previous: 2014-01-01 00:00:00,BiBLEU-Previous: 2014-01-01 00:00:00,UniBLEU-Previous: 2016-01-01 00:00:00,BiBLEU-Previous: 2016-01-01 00:00:00,UniBLEU-Previous: 2018-01-01 00:00:00,BiBLEU-Previous: 2018-01-01 00:00:00
0,100.00,Pauley Perrette --,what is abby's real name from ncis,77.0,Valdosta State University Pauley Perrette,100.0,Pauley Perrette,100.0,Pauley Perrette,100.0,Pauley Perrette,100.0,Pauley Perrette,5401,100.000000,100.0,Pauley Perrette,100.0,Pauley Perrette,100.0,Pauley Perrette,0.0,100.0,1.000000e+02,100.000000,1.000000e+02,100.0,1.000000e+02,100.0,1.000000e+02,100.0,1.000000e+02,100.000000,1.000000e+02,100.0,1.000000e+02,100.0,1.000000e+02,100.000000,1.000000e+02,100.000000,1.000000e+02,100.0,1.000000e+02,100.000000,1.000000e+02
1,100.00,2009 --,what year did flight 1549 land in the hudson,100.0,2009,100.0,2009,100.0,2009,100.0,2009,100.0,2009,8880,100.000000,0.0,No Edit,100.0,2009,100.0,2009,100.0,100.0,2.225074e-306,0.000000,0.000000e+00,100.0,2.225074e-306,100.0,2.225074e-306,100.0,2.225074e-306,100.000000,2.225074e-306,100.0,2.225074e-306,0.0,0.000000e+00,100.000000,2.225074e-306,100.000000,2.225074e-306,100.0,2.225074e-306,100.000000,2.225074e-306
3,100.00,1983 --,when did the red hot chili peppers start,100.0,1983,100.0,1983,100.0,1983,100.0,1983,100.0,1983,5498,100.000000,100.0,1983,100.0,1983,100.0,1983,0.0,100.0,2.225074e-306,100.000000,2.225074e-306,100.0,2.225074e-306,100.0,2.225074e-306,100.0,2.225074e-306,100.000000,2.225074e-306,100.0,2.225074e-306,100.0,2.225074e-306,100.000000,2.225074e-306,100.000000,2.225074e-306,100.0,2.225074e-306,100.000000,2.225074e-306
4,100.00,three --,what is the highest number of michelin stars a resturant can receive,100.0,three,100.0,three,100.0,three,100.0,three,100.0,three,8904,70.666667,12.0,four or five,100.0,three,100.0,three,88.0,100.0,2.225074e-306,0.000000,0.000000e+00,100.0,2.225074e-306,100.0,2.225074e-306,100.0,2.225074e-306,100.000000,2.225074e-306,100.0,2.225074e-306,0.0,0.000000e+00,100.000000,2.225074e-306,100.000000,2.225074e-306,100.0,2.225074e-306,100.000000,2.225074e-306
5,100.00,Kerry King --,who played guitar in no sleep till brooklyn,100.0,Kerry King,100.0,"Kerry King,",100.0,"Kerry King,",100.0,"Kerry King,",100.0,"Kerry King,",5451,100.000000,100.0,"Kerry King,",100.0,"Kerry King,",100.0,"Kerry King,",0.0,50.0,2.225074e-306,50.000000,2.225074e-306,50.0,2.225074e-306,50.0,2.225074e-306,50.0,2.225074e-306,50.000000,2.225074e-306,50.0,2.225074e-306,100.0,1.000000e+02,100.000000,1.000000e+02,100.000000,1.000000e+02,100.0,1.000000e+02,100.000000,1.000000e+02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11262,9.50,the United States Congress --,who creat

In [151]:
sorted(list(bleu_tot.filter(regex="Avg Score").columns))

['Avg Score: 2008-01-01 00:00:00',
 'Avg Score: 2010-01-01 00:00:00',
 'Avg Score: 2012-01-01 00:00:00',
 'Avg Score: 2014-01-01 00:00:00',
 'Avg Score: 2016-01-01 00:00:00',
 'Avg Score: 2018-01-01 00:00:00',
 'Avg Score: original']

In [152]:
def year_sort(df, reg):
    return sorted(list(df.filter(regex=reg).columns))

In [156]:
cols_ordered = []
for reg in ["^Avg Score", "^Predicted", "^UniBLEU-Original", "^BiBLEU-Original", "^UniBLEU-Previous", "^BiBLEU-Previous"]:
    cols_ordered.append(year_sort(bleu_tot, reg))

In [157]:
cols_ordered

[['Avg Score: 2008-01-01 00:00:00',
  'Avg Score: 2010-01-01 00:00:00',
  'Avg Score: 2012-01-01 00:00:00',
  'Avg Score: 2014-01-01 00:00:00',
  'Avg Score: 2016-01-01 00:00:00',
  'Avg Score: 2018-01-01 00:00:00',
  'Avg Score: original'],
 ['Predicted: 2008-01-01 00:00:00',
  'Predicted: 2010-01-01 00:00:00',
  'Predicted: 2012-01-01 00:00:00',
  'Predicted: 2014-01-01 00:00:00',
  'Predicted: 2016-01-01 00:00:00',
  'Predicted: 2018-01-01 00:00:00'],
 ['UniBLEU-Original: 2008-01-01 00:00:00',
  'UniBLEU-Original: 2010-01-01 00:00:00',
  'UniBLEU-Original: 2012-01-01 00:00:00',
  'UniBLEU-Original: 2014-01-01 00:00:00',
  'UniBLEU-Original: 2016-01-01 00:00:00',
  'UniBLEU-Original: 2018-01-01 00:00:00'],
 ['BiBLEU-Original: 2008-01-01 00:00:00',
  'BiBLEU-Original: 2010-01-01 00:00:00',
  'BiBLEU-Original: 2012-01-01 00:00:00',
  'BiBLEU-Original: 2014-01-01 00:00:00',
  'BiBLEU-Original: 2016-01-01 00:00:00',
  'BiBLEU-Original: 2018-01-01 00:00:00'],
 ['UniBLEU-Previous: 2010-01-

In [ ]:
new_cols = []
for i in range(cols_ordered.shape[1])
for i in len(cols_ordered):
    